In [1]:
# Dataprofiler's dataloader with Popmons report system Demo

In [2]:
# imports
import os
import dataprofiler as dp
import pandas as pd
import argparse
import popmon  # noqa

In [3]:
# This is used to mimic the loading of the popmon default dataset 
# Headers are handled in a slightly different way with the flight 
# delay dataset from popmon example
def convert_headers(dataframe):
    new_header = dataframe.iloc[0] #grab the first row for the header
    dataframe = dataframe[1:] #take the data less the header row
    dataframe.columns = new_header #set the header row as the df header
    return dataframe

In [4]:
# Popmon dataloader example (deafult csv)
def pm_dataloader(path, time_index):
    # Load pm dataframe (Can only read csvs unless reader option is changed)
    if not time_index is None:
        pm_data = pd.read_csv(path, parse_dates=[time_index])
    else:
        time_index = True
        pm_data = pd.read_csv(path)
    return pm_data

In [5]:
# Dataprofiler dataloader example
def dp_dataloader(path, time_index):
    # Datalaoder from dataprofiler used
    dp_data = dp.Data(path) 

    # Used to mimic the default columns read in by readcsv function
#     tmp_headers = [header for header in range(len(dp_data.data.columns))]
    dataframe = dp_data.data
#     if dataframe.columns.values.tolist() == tmp_headers:
#         dataframe = convert_headers(dataframe)

    # Make dataframe from dataprofiler popmon compliant
    if not time_index is True:
        # Mimics parse_dates=[time_index] optional param
        dp_data.data[time_index] = pd.to_datetime(dp_data.data[time_index])

    return dp_data.data, time_index

In [6]:
# Default csv from popmon example
path = "../dataprofiler/tests/data/csv/flight_delays.csv"
time_index = "DATE"
report_output_dir = "./popmon_output/flight_delays"


In [7]:
# Random csv from dataprofiler tests
path = "../dataprofiler/tests/data/csv/aws_honeypot_marx_geo.csv"
time_index = "datetime"
report_output_dir = "./popmon_output/aws_honeypot_marx_geo"

In [8]:
# Random json file from dataprofiler tests
path = "../dataprofiler/tests/data/json/math.json"

time_index = "data.9"
report_output_dir = "./popmon_output/math"

In [9]:
# Random avro file from dataprofiler tests
path = "../dataprofiler/tests/data/avro/popmon_data_example_with_datetime.avro"

time_index = "DATETIME0"
report_output_dir = "./popmon_output/popmon_data_example_with_datetime_avro"

In [10]:
# Random parquet file from dataprofiler tests
path = "../dataprofiler/tests/data/parquet/popmon_data_example_with_datetime.parquet"

time_index = "DATETIME0"
report_output_dir = "./popmon_output/popmon_data_example_with_datetime_parquet"

In [11]:
if not os.path.exists(report_output_dir):
    os.makedirs(report_output_dir)

In [12]:
try:
    pm_data = pm_dataloader(path, time_index)
except Exception as e:
    pm_data = None
    print(e)

'utf-8' codec can't decode byte 0xc6 in position 7: invalid continuation byte


In [13]:
dp_dataframe, dp_time_index = dp_dataloader(path, time_index)

2022-01-03 11:37:15,426 INFO [api]: cp037 was excluded because of initial chaos probing. Gave up 2 time(s). Computed mean chaos is 416.400000 %.
2022-01-03 11:37:15,428 INFO [api]: cp1026 is deemed too similar to code page cp037 and was consider unsuited already. Continuing!
2022-01-03 11:37:15,430 INFO [api]: cp1125 was excluded because of initial chaos probing. Gave up 2 time(s). Computed mean chaos is 133.000000 %.
2022-01-03 11:37:15,432 INFO [api]: cp1140 is deemed too similar to code page cp037 and was consider unsuited already. Continuing!
2022-01-03 11:37:15,440 INFO [api]: cp1256 was excluded because of initial chaos probing. Gave up 2 time(s). Computed mean chaos is 129.050000 %.
2022-01-03 11:37:15,443 INFO [api]: cp273 is deemed too similar to code page cp037 and was consider unsuited already. Continuing!
2022-01-03 11:37:15,447 INFO [api]: cp437 was excluded because of initial chaos probing. Gave up 2 time(s). Computed mean chaos is 126.450000 %.
2022-01-03 11:37:15,448 IN

2022-01-03 11:37:15,574 INFO [api]: iso8859_15 is deemed too similar to code page iso8859_10 and was consider unsuited already. Continuing!
2022-01-03 11:37:15,576 INFO [api]: iso8859_16 was excluded because of initial chaos probing. Gave up 1 time(s). Computed mean chaos is 155.000000 %.
2022-01-03 11:37:15,576 INFO [api]: iso8859_2 is deemed too similar to code page iso8859_16 and was consider unsuited already. Continuing!
2022-01-03 11:37:15,577 INFO [api]: iso8859_4 is deemed too similar to code page iso8859_10 and was consider unsuited already. Continuing!
2022-01-03 11:37:15,578 INFO [api]: iso8859_5 was excluded because of initial chaos probing. Gave up 1 time(s). Computed mean chaos is 168.500000 %.
2022-01-03 11:37:15,579 INFO [api]: iso8859_9 is deemed too similar to code page iso8859_10 and was consider unsuited already. Continuing!
2022-01-03 11:37:15,580 INFO [api]: koi8_r was excluded because of initial chaos probing. Gave up 1 time(s). Computed mean chaos is 166.600000 %

2022-01-03 11:37:15,696 INFO [api]: cp500 is deemed too similar to code page cp037 and was consider unsuited already. Continuing!
2022-01-03 11:37:15,696 INFO [api]: cp775 was excluded because of initial chaos probing. Gave up 1 time(s). Computed mean chaos is 166.200000 %.
2022-01-03 11:37:15,697 INFO [api]: cp850 is deemed too similar to code page cp437 and was consider unsuited already. Continuing!
2022-01-03 11:37:15,697 INFO [api]: cp852 was excluded because of initial chaos probing. Gave up 1 time(s). Computed mean chaos is 175.900000 %.
2022-01-03 11:37:15,698 INFO [api]: cp855 was excluded because of initial chaos probing. Gave up 1 time(s). Computed mean chaos is 187.200000 %.
2022-01-03 11:37:15,698 INFO [api]: cp858 is deemed too similar to code page cp437 and was consider unsuited already. Continuing!
2022-01-03 11:37:15,699 INFO [api]: cp860 is deemed too similar to code page cp437 and was consider unsuited already. Continuing!
2022-01-03 11:37:15,699 INFO [api]: cp861 is 

In [14]:
if pm_data is not None:
    report_pm_loader = pm_data.pm_stability_report(
                time_axis=time_index,
                time_width="1w",
                time_offset="2015-07-02",
                extended_report=False,
                pull_rules={"*_pull": [10, 7, -7, -10]},
            )
    # Save pm reports
    report_pm_loader.to_file(os.path.join(report_output_dir, "popmon_loader_report.html"))
    print("Popmon dataloader report generated at:")
    print(os.path.join(report_output_dir, "popmon_loader_report.html"))
else:
    print("Error: Popmon not able to generated report because of invalid data type!")

In [15]:
report_dp_loader = dp_dataframe.pm_stability_report(
            time_axis=dp_time_index,
            time_width="1w",
            time_offset="2015-07-02",
            extended_report=False,
            pull_rules={"*_pull": [10, 7, -7, -10]},
        )
# Save pm reports
report_dp_loader.to_file(os.path.join(report_output_dir, "dataprofiler_loader_report.html"))
print("Dataprofiler dataloader report generated at:")
print(os.path.join(report_output_dir, "popmon_loader_report.html"))

/Users/ksneab/c1_projects/CapitalOneRepos/popmon_with_dp/venv/lib/python3.7/site-packages/histogrammar/dfinterface/make_histograms.py:172: UserWarning: time-axis "DATETIME0" already found in binning specifications. not overwriting.
  f'time-axis "{time_axis}" already found in binning specifications. not overwriting.'
2022-01-03 11:37:15,761 INFO [histogram_filler_base]: Filling 12 specified histograms. auto-binning.
100%|███████████████████████████████████████████████████████████████| 12/12 [00:04<00:00,  2.46it/s]
2022-01-03 11:37:20,637 INFO [hist_splitter]: Splitting histograms "hists" as "split_hists"
2022-01-03 11:37:21,152 INFO [hist_comparer]: Comparing "split_hists" with rolling sum of 1 previous histogram(s).
2022-01-03 11:37:34,186 INFO [hist_comparer]: Comparing "split_hists" with reference "split_hists"
2022-01-03 11:37:46,532 INFO [pull_calculator]: Comparing "comparisons" with median/mad of reference "comparisons"
2022-01-03 11:37:49,174 INFO [hist_profiler]: Profiling hi

Dataprofiler dataloader report generated at:
./popmon_output/popmon_data_example_with_datetime_parquet/popmon_loader_report.html
